In [1]:
#for collab I think you do this to install ffmpeg (not sure verify it works)
#!apt-get update
#!apt-get install -y ffmpeg


In [2]:
#for collab
#!pip install musdb stempeg ffmpeg-python


In [1]:
import os
os.environ["PATH"] = "/opt/homebrew/Cellar/ffmpeg/8.0_2/bin:" + os.environ["PATH"]
import sys
sys.path.append("/Users/ivanarasch/Desktop/GradSchool/MIR/SourceSeparationProject")

In [2]:
import musdb
import stempeg
import ffmpeg
import utils
import importlib
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
import librosa.display
import IPython.display as ipd
from pathlib import Path

In [3]:
mus = musdb.DB(
    root="/Users/ivanarasch/Desktop/GradSchool/MIR/SourceSeparationProject/musdb18",
    subsets="train",  # or "test"
    is_wav=False      
)


In [6]:
len(mus)


100

In [4]:
track = mus[0]
print(track)


A Classic Education - NightOwl


In [5]:
track.targets.keys()

odict_keys(['vocals', 'drums', 'bass', 'other', 'accompaniment', 'linear_mixture'])

In [4]:
import pandas as pd

mus = musdb.DB(root="musdb18", is_wav=False)

rows = []

for track in mus:
    track_name = track.name
    stems = list(track.targets.keys())
    rows.append({
        "track_name": track_name,
        "stems": ", ".join(stems),
        "num_stems": len(stems)
    })

df = pd.DataFrame(rows)
df

,track_name,stems,num_stems
0,A Classic Education - NightOwl,"vocals, drums, bass, other, accompaniment, lin...",6
1,ANiMAL - Clinic A,"vocals, drums, bass, other, accompaniment, lin...",6
2,ANiMAL - Easy Tiger,"vocals, drums, bass, other, accompaniment, lin...",6
3,ANiMAL - Rockshow,"vocals, drums, bass, other, accompaniment, lin...",6
4,Actions - Devil's Words,"vocals, drums, bass, other, accompaniment, lin...",6
...,...,...,...
145,Timboz - Pony,"vocals, drums, bass, other, accompaniment, lin...",6
146,Tom McKenzie - Directions,"vocals, drums, bass, other, accompaniment, lin...",6
147,Triviul feat. The Fiend - Widow,"vocals, drums, bass, other, accompaniment, lin...",6
148,We Fell From The Sky - Not You,"vocals, drums, bass, other, accompaniment, lin...",6


In [10]:
#if you want to export a csv and look at data 
#df.to_csv("musdb_stem_list.csv", index=False) 

In [5]:
root = "/Users/ivanarasch/Desktop/GradSchool/MIR/SourceSeparationProject/musdb18"

mus = musdb.DB(root=root, subsets="train")

track = mus[0]       # pick the first song
print(track.name)    # see which one it is


A Classic Education - NightOwl


Experiment 1 Interferance in Vocal Separiation

In [8]:
vocals = track.targets['vocals'].audio
drums = track.targets['drums'].audio
accompaniment = track.targets ['accompaniment'].audio
bass = track.targets['bass'].audio
other = track.targets['other'].audio

In [13]:
drums_vocals = drums + vocals
accompaniment_vocals = accompaniment + vocals
bass_vocals = bass + vocals
other_vocals = other + vocals



In [14]:
import soundfile as sf

sf.write("drums_vocals.wav", drums_vocals, track.rate)
sf.write("accompaniment_vocals.wav", accompaniment_vocals, track.rate)
sf.write("bass_vocals.wav", bass_vocals, track.rate)
sf.write("other_vocals.wav", other_vocals, track.rate)



Experiment 3: Effect of Vocal FX on Separation Robustness

In [6]:
REVERB_LEVELS = [
    {"delay_ms": 30, "repeats": 2},
    {"delay_ms": 50, "repeats": 3},
    {"delay_ms": 80, "repeats": 4},
    {"delay_ms": 120, "repeats": 5},
]

DELAY_LEVELS = [
    {"delay_ms": 120, "repeats": 1},
    {"delay_ms": 200, "repeats": 2},
    {"delay_ms": 300, "repeats": 3},
    {"delay_ms": 400, "repeats": 4},
]

CHORUS_LEVELS = [
    {"depth": 0.1, "rate": 0.2, "wet": 0.10},
    {"depth": 0.3, "rate": 0.5, "wet": 0.25},
    {"depth": 0.6, "rate": 1.0, "wet": 0.45},
    {"depth": 1.2, "rate": 2.5, "wet": 0.70},
]

BITCRUSH_LEVELS = [
    {"bit_depth": 12, "downsample_factor": 1},
    {"bit_depth": 8, "downsample_factor": 2},
    {"bit_depth": 6, "downsample_factor": 4},
    {"bit_depth": 4, "downsample_factor": 8},
]

COMPRESSION_LEVELS = [
    {"threshold": -20, "ratio": 2.0},
    {"threshold": -25, "ratio": 3.0},
    {"threshold": -30, "ratio": 4.0},
    {"threshold": -35, "ratio": 6.0},
]


In [7]:
def apply_effect(vocals_np, rate, effect_func, params):
    vocal_audio = utils.audiosegment_from_numpy(vocals_np, rate)
    fx_vocals = effect_func(vocal_audio, **params)
    return utils.audiosegment_to_numpy(fx_vocals)

In [8]:
mus = musdb.DB(root="musdb18", subsets="test")

OUTPUT_ROOT = Path("musdb18synthesized")
OUTPUT_ROOT.mkdir(exist_ok=True)

for track in mus.tracks:
    print(f"\n=== Processing {track.name} ===")

    track_dir = OUTPUT_ROOT / track.name
    (track_dir / "original").mkdir(parents=True, exist_ok=True)

    rate = track.rate
    mix = track.audio

    vocals = track.targets["vocals"].audio
    bass = track.targets["bass"].audio
    drums = track.targets["drums"].audio
    other = track.targets["other"].audio

    # Save original stems + mix
    sf.write(track_dir / "original" / "mix.wav", mix, rate)
    sf.write(track_dir / "original" / "vocals.wav", vocals, rate)
    sf.write(track_dir / "original" / "bass.wav", bass, rate)
    sf.write(track_dir / "original" / "drums.wav", drums, rate)
    sf.write(track_dir / "original" / "other.wav", other, rate)

    # =============================
    # Apply each effect
    # =============================

    # Reverb
    for i, params in enumerate(REVERB_LEVELS, 1):
        out = track_dir / f"reverb_l{i}"
        out.mkdir(exist_ok=True)

        fx_vocals = apply_effect(vocals, rate, utils.add_reverb, params)

        fx_mix = mix.copy()
        fx_mix[:len(vocals)] += fx_vocals - vocals
        fx_mix = fx_mix.clip(-1, 1)

        sf.write(out / "mix.wav", fx_mix, rate)

    # Delay
    for i, params in enumerate(DELAY_LEVELS, 1):
        out = track_dir / f"delay_l{i}"
        out.mkdir(exist_ok=True)

        fx_vocals = apply_effect(vocals, rate, utils.add_delay, params)

        fx_mix = mix.copy()
        fx_mix[:len(vocals)] += fx_vocals - vocals
        fx_mix = fx_mix.clip(-1, 1)

        sf.write(out / "mix.wav", fx_mix, rate)

    # Chorus
    for i, params in enumerate(CHORUS_LEVELS, 1):
        out = track_dir / f"chorus_l{i}"
        out.mkdir(exist_ok=True)

        fx_vocals = apply_effect(vocals, rate, utils.add_chorus, params)
        fx_mix = mix.copy()
        fx_mix[:len(vocals)] += fx_vocals - vocals
        fx_mix = fx_mix.clip(-1, 1)

        sf.write(out / "mix.wav", fx_mix, rate)

    # Bitcrush
    for i, params in enumerate(BITCRUSH_LEVELS, 1):
        out = track_dir / f"bitcrush_l{i}"
        out.mkdir(exist_ok=True)

        fx_vocals = apply_effect(vocals, rate, utils.add_bitcrush, params)

        fx_mix = mix.copy()
        fx_mix[:len(vocals)] += fx_vocals - vocals
        fx_mix = fx_mix.clip(-1, 1)

        sf.write(out / "mix.wav", fx_mix, rate)

    # Compression
    for i, params in enumerate(COMPRESSION_LEVELS, 1):
        out = track_dir / f"compression_l{i}"
        out.mkdir(exist_ok=True)

        fx_vocals = apply_effect(vocals, rate, utils.add_compression, params)

        fx_mix = mix.copy()
        fx_mix[:len(vocals)] += fx_vocals - vocals
        fx_mix = fx_mix.clip(-1, 1)

        sf.write(out / "mix.wav", fx_mix, rate)



=== Processing AM Contra - Heart Peripheral ===


TypeError: add_reverb() got an unexpected keyword argument 'delay_ms'